# Reveiwing dataset 

In reviewing the dataset, we've identified columns that appear to be irrelevant for our analysis. The columns "Message ID," "Message timestamp," "Message date," and "Message time" seem to hold information related to when the data is entered, which is not crucial for our purposes.

Additionally, columns such as "Start time," "Start time," "End time," "End time," and "Catch area" will be replaced with a consolidated "Duration" column. However, we acknowledge that the time of day might still be relevant for understanding fish behavior.

We've opted to use "Location start" and "Location stop" instead of "Main area start," "Main area end," "Main area start (code)," and "Main area end (code)" since all these columns seem to be interconnected.

Upon further examination, we noticed that "Location start (code)" and "Location stop (code)" contained numerous empty fields. To address this, we recommend using start position latitude/longitude and stop position latitude/longitude, which should provide more meaningful data compared to the (code) fields, likely resulting in higher precision.

Another issue arises with the "Gross tonnage" column, where a significant number (approximately 230,000) of entries are missing. Consequently, we have decided to drop both "Gross tonnage" columns in favor of relying on boat length and width, which offer more comprehensive information.

Similarly, we are simplifying the dataset by using only the "Vessel length" column instead of both 'Length group' and 'Length group (code)'. Although this may introduce some complexity in categorizing various vessel types, it should contribute to better precision in our analysis.

Finally, to streamline the data further, we have chosen to focus on the "Art FAO" column, removing all other species types for a more targeted and manageable dataset.

In [8]:
#Hvilke kolonner er irrelevante?
#"Melding ID", "Meldingstidpunkt", "Meldingsdato", "Meldingsklokkeslett" Det meste med når dette er lagt inn vil ikke ha noe å si.
#"Starttidpunkt", "Startklokkeslett", "Stopptidspunkt", "Stoppklokkeslett", "Fangstår" Vil erstattes med bare "Varighet", men kanskje har noe å si med åssen fisken oppfører seg natt/dag.
#"Hovedområde start", "Hovedområde stopp", "Hovedområde start (kode)", "Hovedområde stopp (kode)" Bruker heller Lokasjon stopp og start ettersom alle de ser ut til å være tilknyttet.
"""
Når det i tillegg sjekket for NaN lenger nede i koden, ser man at "Lokasjon start (kode)" og "Lokasjon stopp (kode)" hadde flere tomme felter. Derfor vil
det være mer gunstig og bruke startposisjon bredde/lengde og samme med stopp. Dette vil gir mer data enn å bruke (kode), men vil trolig og gi en høyere 
presisjon.
"""
#Problem med også "Bruttotonnasje" med veldig mange(ca 230000) NaN derfor droppe begge "Bruttotonnasje", ettersom vi har mange fler med båt lengde og bredde.
#Eksempelhvis samme med 'Lengdegruppe' og 'Lengdegruppe (kode)' brukes bare 'Fartøylengde'. Kan gi flere ulike farttøy som blir mer komplisert data, men bidra bedre presisjon
#Alt av unødvendig art, vi valgte Art FAO og fjernet alle andre art typer.


'\nNår det i tillegg sjekket for NaN lenger nede i koden, ser man at "Lokasjon start (kode)" og "Lokasjon stopp (kode)" hadde flere tomme felter. Derfor vil\ndet være mer gunstig og bruke startposisjon bredde/lengde og samme med stopp. Dette vil gir mer data enn å bruke (kode), men vil trolig og gi en høyere \npresisjon.\n'

# Impoting and reading 
Reading and printing out the first 5 lines 
Using sep=';' because in the csv file it is split with ";"

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv', sep=';')
df.head()

,Melding ID,Meldingstidspunkt,Meldingsdato,Meldingsklokkeslett,Starttidspunkt,Startdato,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),...,Art - FDIR,Art - gruppe (kode),Art - gruppe,Rundvekt,Lengdegruppe (kode),Lengdegruppe,Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde
0,1497177,01.01.2018,01.01.2018,00:00,31.12.2017,31.12.2017,00:00,"-60,35","-46,133",NaN,...,Antarktisk krill,506.0,Antarktisk krill,706714.0,5.0,28 m og over,9432.0,NaN,"19,87","133,88"
1,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Hyse,202.0,Hyse,9594.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
2,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Torsk,201.0,Torsk,8510.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
3,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Blåkveite,301.0,Blåkveite,196.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
4,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Sei,203.0,Sei,134.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"


# Checking for NaN (Missing Values):

When we check for NaN values we can se the context between the colons. With this knowlage we can easaly see if we should drop the rows with NaN. We also see what colons that are worth using

In [10]:

nan_count = df.isna().sum()

print(nan_count)

#Definere irrelevante kolonner sånn at lettere å legge til og slette
irrelevante_col = ['Melding ID', 'Meldingstidspunkt', 'Meldingsdato', 'Meldingsklokkeslett','Starttidspunkt', 'Startklokkeslett', 'Stoppdato',
                   'Stopptidspunkt', 'Stoppklokkeslett','Hovedområde start', 'Hovedområde stopp', 'Hovedområde start (kode)', 'Fangstår',
                   'Hovedområde stopp (kode)', 'Lokasjon start (kode)', 'Lokasjon stopp (kode)', 'Bruttotonnasje 1969', 'Bruttotonnasje annen',
                   'Lengdegruppe (kode)', 'Lengdegruppe', 'Hovedart FAO (kode)', 'Hovedart FAO', 'Hovedart - FDIR (kode)', 'Art FAO (kode)', 
                   'Art - FDIR (kode)', 'Art FAO', 'Art - gruppe (kode)', 'Art - gruppe', 'Redskap FAO (kode)', 'Redskap FAO', 'Redskap FDIR']

#Removing the coloms that are unneceasrry with pandas drop 
for col in irrelevante_col:
     #Med bruk av axis=1 settes det sånn at den dropper kolonner
     df = df.drop(col, axis=1)
     
#Dropper NaN istede for eksmepelhvis å erstatte mean ettersom ikke bruke mean på eksempel hvis redskap og koder
#Vi dropper også NaN fordi er mindre enn 5000 NaN av resterende kolonner, og vi vil heller ha en klarere data

df.dropna()

df.head()

Melding ID                       0
Meldingstidspunkt                0
Meldingsdato                     0
Meldingsklokkeslett              0
Starttidspunkt                   0
Startdato                        0
Startklokkeslett                 0
Startposisjon bredde             0
Startposisjon lengde             0
Hovedområde start (kode)      2001
Hovedområde start             4124
Lokasjon start (kode)         2001
Havdybde start                   0
Stopptidspunkt                   0
Stoppdato                        0
Stoppklokkeslett                 0
Varighet                         0
Fangstår                         0
Stopposisjon bredde              0
Stopposisjon lengde              0
Hovedområde stopp (kode)      1962
Hovedområde stopp             4124
Lokasjon stopp (kode)         1962
Havdybde stopp                   0
Trekkavstand                    24
Redskap FAO (kode)               0
Redskap FAO                    188
Redskap FDIR (kode)            188
Redskap FDIR        

,Startdato,Startposisjon bredde,Startposisjon lengde,Havdybde start,Varighet,Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Redskap FDIR (kode),Art - FDIR,Rundvekt,Bredde,Fartøylengde
0,31.12.2017,"-60,35","-46,133",-2740,1440,"-60,4","-46,15",-805,5649.0,53.0,Antarktisk krill,706714.0,"19,87","133,88"
1,30.12.2017,"74,885","16,048",-335,295,"74,914","15,969",-334,3970.0,51.0,Hyse,9594.0,"12,6","56,8"
2,30.12.2017,"74,885","16,048",-335,295,"74,914","15,969",-334,3970.0,51.0,Torsk,8510.0,"12,6","56,8"
3,30.12.2017,"74,885","16,048",-335,295,"74,914","15,969",-334,3970.0,51.0,Blåkveite,196.0,"12,6","56,8"
4,30.12.2017,"74,885","16,048",-335,295,"74,914","15,969",-334,3970.0,51.0,Sei,134.0,"12,6","56,8"


In [11]:
counts = df['Art - FDIR'].value_counts()

print(counts)

Torsk                  56574
Sei                    42557
Hyse                   39120
Lange                  17338
Uer (vanlig)           14153
                       ...  
Sagtang                    1
Annen makrell              1
Blåhval                    1
Annen laksefisk            1
Bukstripet pelamide        1
Name: Art - FDIR, Length: 122, dtype: int64


In [12]:
df = df[(df['Art - FDIR'] == 'Torsk') | (df['Art - FDIR'] == 'Sei')]
df.head()

,Startdato,Startposisjon bredde,Startposisjon lengde,Havdybde start,Varighet,Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Redskap FDIR (kode),Art - FDIR,Rundvekt,Bredde,Fartøylengde
2,30.12.2017,"74,885","16,048",-335,295,"74,914","15,969",-334,3970.0,51.0,Torsk,8510.0,"12,6","56,8"
4,30.12.2017,"74,885","16,048",-335,295,"74,914","15,969",-334,3970.0,51.0,Sei,134.0,"12,6","56,8"
6,31.12.2017,"74,91","15,868",-403,267,"74,901","16,248",-277,11096.0,51.0,Torsk,6651.0,"12,6","56,8"
9,31.12.2017,"74,91","15,868",-403,267,"74,901","16,248",-277,11096.0,51.0,Sei,67.0,"12,6","56,8"
12,31.12.2017,"74,883","16,056",-346,315,"74,924","15,742",-496,10215.0,51.0,Torsk,5097.0,"12,6","56,8"


In [13]:
float_columns = ['Startposisjon bredde', 'Startposisjon lengde', 'Havdybde start', 'Varighet',
                'Stopposisjon bredde', 'Stopposisjon lengde', 'Havdybde stopp', 'Trekkavstand', 'Redskap FDIR (kode)', 
                'Rundvekt', 'Bredde', 'Fartøylengde'] 
# Convert the relevant columns to string
df[float_columns] = df[float_columns].astype(str)

# Replace , with . for all float columns
for col in float_columns:
    df[col] = df[col].str.replace(',', '.')

# Convert the columns back to float
df[float_columns] = df[float_columns].astype(float)
mapping = {'Torsk':  0, 'Sei':  1}
df['Art - FDIR'] = df['Art - FDIR'].map(mapping)
df.head()



,Startdato,Startposisjon bredde,Startposisjon lengde,Havdybde start,Varighet,Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Redskap FDIR (kode),Art - FDIR,Rundvekt,Bredde,Fartøylengde
2,30.12.2017,74.885,16.048,-335.0,295.0,74.914,15.969,-334.0,3970.0,51.0,0,8510.0,12.6,56.8
4,30.12.2017,74.885,16.048,-335.0,295.0,74.914,15.969,-334.0,3970.0,51.0,1,134.0,12.6,56.8
6,31.12.2017,74.910,15.868,-403.0,267.0,74.901,16.248,-277.0,11096.0,51.0,0,6651.0,12.6,56.8
9,31.12.2017,74.910,15.868,-403.0,267.0,74.901,16.248,-277.0,11096.0,51.0,1,67.0,12.6,56.8
12,31.12.2017,74.883,16.056,-346.0,315.0,74.924,15.742,-496.0,10215.0,51.0,0,5097.0,12.6,56.8


In [14]:
#Dato må endres til "Values" for eksempel 0,1,2,3 for vinter, vår, høst og, sommer
#Også vil du da kunne bruke alle column istede for legge til features
df['Startdato'] = pd.to_datetime(df['Startdato'], format='%d.%m.%Y')
df['Month'] = df['Startdato'].dt.month
df.drop('Startdato', axis=1)

,Startposisjon bredde,Startposisjon lengde,Havdybde start,Varighet,Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Redskap FDIR (kode),Art - FDIR,Rundvekt,Bredde,Fartøylengde,Month
2,74.885,16.048,-335.0,295.0,74.914,15.969,-334.0,3970.0,51.0,0,8510.0,12.6,56.8,12
4,74.885,16.048,-335.0,295.0,74.914,15.969,-334.0,3970.0,51.0,1,134.0,12.6,56.8,12
6,74.910,15.868,-403.0,267.0,74.901,16.248,-277.0,11096.0,51.0,0,6651.0,12.6,56.8,12
9,74.910,15.868,-403.0,267.0,74.901,16.248,-277.0,11096.0,51.0,1,67.0,12.6,56.8,12
12,74.883,16.056,-346.0,315.0,74.924,15.742,-496.0,10215.0,51.0,0,5097.0,12.6,56.8,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305408,76.509,14.295,-193.0,301.0,76.431,14.679,-341.0,13284.0,51.0,0,3126.0,12.6,57.3,12
305413,76.509,14.295,-193.0,301.0,76.431,14.679,-341.0,13284.0,51.0,1,32.0,12.6,57.3,12
305416,76.385,14.932,-244.0,345.0,76.640,13.642,-299.0,44035.0,51.0,0,3452.0,12.6,57.3,12
305422,76.606,13.800,-280.0,328.0,76.899,12.820,-289.0,41216.0,51.0,0,5902.0,12.6,57.3,12
